In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,062 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:13 http://arch

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-14 17:25:17--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.83MB/s    in 0.2s    

2023-02-14 17:25:18 (4.83 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...|2015-01-31 00:08:00|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...| 

# Transform the Data

In [5]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

9002021
9001052
9001052


In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create the "review_id_table".

In [7]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R100A9LP02ZIKS|    6134747|B003TUSWG2|     443574173|2013-01-26 00:12:00|
|R100BSJ3U71OX3|   33540279|B004IEWXSG|     425207258|2013-01-31 00:12:00|
|R100CN4258RRXR|   13193493|B00DCIZJ5S|     706124906|2013-01-24 00:12:00|
|R100R8KE8VENBT|   27990716|B00ANJL72I|     742348004|2013-01-12 00:01:00|
|R100VZJSXIER38|   43156836|B0095ZCPTK|     302992785|2013-01-06 00:03:00|
|R100YZDLL0O2N4|   12804693|B008ODNMD2|     675633080|2014-01-21 00:02:00|
|R101C0M426BM0Z|    6968110|B005X1Y7I2|     908777325|2014-01-06 00:09:00|
|R101EF8XBZDT73|    2996829|B00FM7R1WG|     930571712|2013-01-30 00:12:00|
|R101F1O5T0PK5M|   24836939|B003ZX7RL4|     245455048|2011-01-17 00:03:00|
|R101IMOK0JKOQW|   11072936|B00OPV34E4|     568061719|2015-01-30 00:05:00|
|R101KUBDDFZSUZ|   378914

## Create the "products" Table

In [8]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id","product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B003TUSWG2|DragonPad USB Car...|
|B004IEWXSG|Cover For Iphone ...|
|B00DCIZJ5S|Galaxy Note 5 Cas...|
|B00ANJL72I|WHITE / WHITE Sho...|
|B0095ZCPTK|Splash Zero Ultra...|
|B008ODNMD2|elago G5 Breathe ...|
|B005X1Y7I2|Anker PowerCore+ ...|
|B00FM7R1WG|ZTE Valet Android...|
|B003ZX7RL4|Nokia N8 Unlocked...|
|B00OPV34E4|Bell+Howell Cleve...|
|B00C53YPHY|Everydaysource Co...|
|B00L8CHL4C|Aimo Wireless For...|
|B00IJNFJ4W|4 items Combo: Mi...|
|B006FZ1SVO|ERJFGE(TM) Rugged...|
|B000UOD5QM|GARMIN 010-10997-...|
|B000PDUZIE|Garmin eTrex Summ...|
|B00EOYOF36|Yubi Power USB Ca...|
|B004JRY402|iClever Wireless ...|
|B004CN9UG6|Trident Case Krak...|
|B00INMWRDU|For Samsung Galax...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [9]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48040012|             1|
|   49227916|             4|
|     285987|             2|
|    2531111|             1|
|    2757697|             1|
|   51937936|             2|
|   44047606|             1|
|   20227024|             9|
|   27682601|             4|
|   36896463|             2|
|   51073768|             1|
|   19035609|             2|
|   10217380|             8|
|   18600073|             3|
|   30205378|             1|
|   42386364|             1|
|   10456237|             1|
|   28805032|             1|
|   12185324|             1|
|   35339821|             1|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [11]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 

vine_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])
vine_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100A9LP02ZIKS|          5|            0|          0|   N|
|R100BSJ3U71OX3|          2|            0|          0|   N|
|R100CN4258RRXR|          5|            0|          0|   N|
|R100R8KE8VENBT|          5|            0|          0|   N|
|R100VZJSXIER38|          5|            0|          0|   N|
|R100YZDLL0O2N4|          5|            0|          0|   N|
|R101C0M426BM0Z|          2|            0|          0|   N|
|R101EF8XBZDT73|          5|            2|          2|   N|
|R101F1O5T0PK5M|          5|            3|          3|   N|
|R101IMOK0JKOQW|          5|            2|          2|   N|
|R101KUBDDFZSUZ|          5|            0|          0|   N|
|R101NE9XX785C4|          4|            0|          0|   N|
|R101V378PT1MG3|          1|            0|          0|   N|
|R1027H9W77VYE0|          5|            

# Load

In [12]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_df', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_df', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_df', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_df', mode=mode, properties=config)